In [2]:
import sympy as sp

# Pretty printing in notebooks
sp.init_printing(use_unicode=True)

# %% [markdown]
# ## Symbols and indexed control points

# %%
t = sp.symbols('t', real=True)
k = sp.symbols('k', integer=True)

# Indexed bases for x/y components
Qx = sp.IndexedBase('Qx')
Qy = sp.IndexedBase('Qy')
P1x = sp.IndexedBase('P1x')
P1y = sp.IndexedBase('P1y')

# %% [markdown]
# ## Build the 2D control points for segment k

# %%
P0  = sp.Matrix([Qx[k],    Qy[k]])          # Q_k
P1k = sp.Matrix([P1x[k],   P1y[k]])         # P1_k
P2  = sp.Matrix([P1x[k+1], P1y[k+1]])       # P1_{k+1}
P3  = sp.Matrix([Qx[k+1],  Qy[k+1]])        # Q_{k+1}

P0, P1k, P2, P3


⎛⎡Qx[k]⎤  ⎡P1x[k]⎤  ⎡P1x[k + 1]⎤  ⎡Qx[k + 1]⎤⎞
⎜⎢     ⎥, ⎢      ⎥, ⎢          ⎥, ⎢         ⎥⎟
⎝⎣Qy[k]⎦  ⎣P1y[k]⎦  ⎣P1y[k + 1]⎦  ⎣Qy[k + 1]⎦⎠

In [18]:
P2

⎡P1x[k + 1]⎤
⎢          ⎥
⎣P1y[k + 1]⎦

In [3]:
# ## Define cubic Bézier segment \(B_k(t)\) as a vector function

# %%
B = (1 - t)**3 * P0 \
  + 3*(1 - t)**2 * t * P1k \
  + 3*(1 - t) * t**2 * P2 \
  + t**3 * P3

B

⎡ 3              2                                   2                 3      ⎤
⎢t ⋅Qx[k + 1] + t ⋅(3 - 3⋅t)⋅P1x[k + 1] + 3⋅t⋅(1 - t) ⋅P1x[k] + (1 - t) ⋅Qx[k]⎥
⎢                                                                             ⎥
⎢ 3              2                                   2                 3      ⎥
⎣t ⋅Qy[k + 1] + t ⋅(3 - 3⋅t)⋅P1y[k + 1] + 3⋅t⋅(1 - t) ⋅P1y[k] + (1 - t) ⋅Qy[k]⎦

In [4]:
# ## Compute second derivative \(B_k''(t)\)

# %%
Bpp = sp.diff(B, t, 2)
Bpp

⎡6⋅(-2⋅t⋅P1x[k + 1] + t⋅P1x[k] + t⋅Qx[k + 1] - (t - 1)⋅P1x[k + 1] + 2⋅(t - 1)⋅ ↪
⎢                                                                              ↪
⎣6⋅(-2⋅t⋅P1y[k + 1] + t⋅P1y[k] + t⋅Qy[k + 1] - (t - 1)⋅P1y[k + 1] + 2⋅(t - 1)⋅ ↪

↪ P1x[k] - (t - 1)⋅Qx[k])⎤
↪                        ⎥
↪ P1y[k] - (t - 1)⋅Qy[k])⎦

In [5]:
# ## Squared Euclidean norm \(\|B_k''(t)\|^2\)
# 
# Use dot product (no square roots, no Abs).

# %%
sqnorm = sp.simplify(Bpp.dot(Bpp))
sqnorm

In [6]:
# ## Integrate over \([0,1]\) to get bending energy \(I_k\)

# %%
Ik = sp.simplify(sp.integrate(sqnorm, (t, 0, 1)))
Ik

In [7]:
# ## Optional: rewrite in a cleaner geometric form
# 
# Differences between consecutive control points often simplify the result.

# %%
D0 = sp.simplify(P1k - P0)
D1 = sp.simplify(P2  - P1k)
D2 = sp.simplify(P3  - P2)

D0, D1, D2

⎛⎡P1x[k] - Qx[k]⎤  ⎡P1x[k + 1] - P1x[k]⎤  ⎡-P1x[k + 1] + Qx[k + 1]⎤⎞
⎜⎢              ⎥, ⎢                   ⎥, ⎢                       ⎥⎟
⎝⎣P1y[k] - Qy[k]⎦  ⎣P1y[k + 1] - P1y[k]⎦  ⎣-P1y[k + 1] + Qy[k + 1]⎦⎠

In [8]:
# Try expressing Ik in terms of differences
Ik_diff = sp.simplify(Ik.rewrite(sp.Add))
Ik_diff

In [10]:
def bending_energy_segment_2d(k_value=None, subs=None):
    """
    Return symbolic I_k.
    - k_value: optionally substitute a concrete integer for k
    - subs: dict of further substitutions (numeric or symbolic)
    """
    expr = Ik
    if k_value is not None:
        expr = expr.subs(k, k_value)
    if subs is not None:
        expr = expr.subs(subs)
    return sp.simplify(expr)

# Example usage (still symbolic):
L = bending_energy_segment_2d()

In [11]:
L

In [ ]:
L.diff(

In [14]:
dL_dP1xk = sp.diff(L, P1x[k])
dL_dP1yk = sp.diff(L, P1y[k])

In [15]:
dL_dP1xk

In [16]:
dL_dP1yk 

In [20]:
sp.diff(L, P1x[k+1])

In [21]:
# Compute bending energy L
L = bending_energy_segment_2d()

# Gradient of L w.r.t. P1_k (x and y components separately)
dL_dP1xk = sp.diff(L, P1x[k])
dL_dP1yk = sp.diff(L, P1y[k])

grad_P1k = sp.Matrix([dL_dP1xk, dL_dP1yk])
grad_P1k


⎡-36⋅P1x[k + 1] + 72⋅P1x[k] - 36⋅Qx[k]⎤
⎢                                     ⎥
⎣-36⋅P1y[k + 1] + 72⋅P1y[k] - 36⋅Qy[k]⎦

In [24]:
bending_energy_segment_2d(k_value=1)

In [ ]:



# %% [markdown]
# ## Helper: function to compute \(I_k\) for any specific k or with substitutions

# %%


# %% [markdown]
# ## Example numeric substitution (optional sanity check)
# 
# Uncomment and edit values if you want to test with numbers.

# %%
# example_subs = {
#     Qx[0]: 0, Qy[0]: 0,
#     P1x[0]: 1, P1y[0]: 2,
#     P1x[1]: 3, P1y[1]: 3,
#     Qx[1]: 4, Qy[1]: 0,
# }
# bending_energy_segment_2d(k_value=0, subs=example_subs).evalf()
